<a href="https://colab.research.google.com/github/sarathklonappan/Unsupervised-Project/blob/main/Unsupervised_BBC_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - TOPIC MODELLING ON NEWS ARTICLES



##### **Project Type**    - Unsupervised
##### **Contribution**    - Team
##### **Team Member 1 -**Sarath Lonappan
##### **Team Member 2 -**Akash Raj


# **Project Summary -**

The dataset contains several articles belonging to diffrent topics.We must select most optimum number of topics and sort the articles to each topic.We have used count vectoriser to convert it to matrix and Latent Dirchlet Allocation algorithm to sort them.The articles have been allocated to respective topics after the modelling.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


In this project, the aim is to find the major topics across a collection of BBC news articles.Categorise each article to main topics using LDA.

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
!pip uninstall scikit-learn

In [ ]:
!pip install scikit-learn==1.0.2

In [ ]:
# installing pyLDAvis to visualize the results of LDA model
!pip install pyLDAvis==2.1.2

In [ ]:
# Import Libraries
# importing os module
import os

# importing CountVectorizer for feature extraction
from sklearn.feature_extraction.text import CountVectorizer

# Importing data manipulation libraries
import numpy as np
import pandas as pd

# importing tqdm and display modules for progress meters/bars
from IPython.display import display
from tqdm import tqdm

# importing wordcloud to represent topics wordcloud
from wordcloud import WordCloud

# Model selection modules
from sklearn.model_selection import GridSearchCV

# Importing Counter
from collections import Counter

# import ast(abstract syntax tree)
import ast

# importing data visualization modules
import matplotlib.pyplot as plt
import seaborn as sns

# importing mlab for implementing MATLAB functions
import matplotlib.mlab as mlab

# importing statistics module
import scipy.stats as stats

# importing TSNE for data exploration and visualizing high-dimensional data
from sklearn.manifold import TSNE

# importing decomposition modules
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation

# importing Natural Language Toolkit and other NLP modules
import nltk
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob, Word

# importing countVectorizer for text vectorization
from sklearn.feature_extraction.text import CountVectorizer

# Importing warnings library. The warnings module handles warnings in Python.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sklearn
sklearn.__version__

### Dataset Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# Load Dataset

# The variable "directory" holds the address of text files stored in drive
path_dir = '/content/drive/MyDrive/M4/Unsupervised project'

# All 5 sub-categories provided
sub_dir = ['business', 'entertainment', 'politics', 'sport', 'tech']

# Create dataframe for gathering the articles
bbc_df = pd.DataFrame()

# Iterate over sub-directories to access the text files
for path in sub_dir:

  # address to the subdirectory
  dir = path_dir + '/' + path

  # Iterate over all the text files present in a sub-directory
  for filename in os.listdir(dir):

    # Get file address
    filepath = os.path.join(dir, filename)

    # Traversing over text files and storing the articles into the dataframe
    try:
      data = open(filepath,'r').read()

      # escape characters to be ignored in the text
      escape = ['\n']

      # removing escape characters from text
      for elem in escape:
        data = data.replace(elem, ' ')

      # Storing article to the dataframe
      dict1 = {'Filename': filename.split('.')[0], 'Article': data.lower(), 'Category':path}
      bbc_df = bbc_df.append(dict1, ignore_index=True, verify_integrity = True)

    # Ignore exception, if any
    except:
      pass

### Dataset First View

In [ ]:
# Dataset First Look
bbc_df.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
bbc_df.shape

### Dataset Information

In [ ]:
# Dataset Info
bbc_df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
bbc_df[bbc_df.duplicated(subset = "Article")]

In [ ]:
# Dropping duplicate entries
bbc_df = bbc_df.drop_duplicates(subset=['Article'], keep='first')
bbc_df

In [ ]:
bbc_df.shape

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count

From info we found that no null values are present in this data frame

### What did you know about your dataset?

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
bbc_df.columns

In [ ]:
# Dataset Describe
bbc_df.describe()

### Variables Description

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
df=pd.DataFrame()

In [ ]:

# Grouping rows based on categories and counting number of entries
df=bbc_df.Category.value_counts()

In [ ]:
df

In [ ]:
# Plotting a bar graph to represent all categories and number of articles in them
df.plot(x = "Category", kind = "bar",
        figsize=(8,5), grid = True,
        xlabel = "Categories",
        ylabel = "Number of Articles",
        colormap= "cividis", width = 0.6,
        rot = 45).patch.set_facecolor('#f0ffff')

# **Articles**

In [ ]:
# Reindexing the data and extracting just the contents of the article i.e, the complete transcript of the article

# Reindexing the data
reindexed_data = bbc_df.reset_index()

# Extracting contents of all articles
reindexed_data = reindexed_data["Article"]

# Display
reindexed_data

# **CountVectorizer (Top Words)**


In [ ]:
# Function to extract top n words with highest frequency
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    The function returns a tuple of the top n words in a sample and their
    accompanying counts, given a CountVectorizer object and text sample as inputs
    '''
    # encoding the document using countvectorizer object
    vectorized_content = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_content, axis=0)

    # extracting specifics for words
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)

    # creating a vector matrix for words
    word_vectors = np.zeros((n_top_words, vectorized_content.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    # display Vector matrix
    print(word_vectors)

    # collect the words
    words = [word[0].encode('ascii').decode('utf-8') for
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])


# Removing Stopwords using NLTK (Natural Language Toolkit)

In [ ]:
# dowloading nltk stopwords module
nltk.download('stopwords')

In [ ]:
# extracting all stopwords for english language
stpwrd = nltk.corpus.stopwords.words('english')

# list of stopwords in english language
stpwrd

In [ ]:
# creating vectorizer object
count_vectorizer = CountVectorizer(stop_words=stpwrd)

# calling the function to get words and their counts
words, word_values = get_top_n_words(n_top_words=25,
                                     count_vectorizer=count_vectorizer,
                                     text_data=reindexed_data)

# display top 25 words using bar plot
fig, ax = plt.subplots(figsize=(16,8))
ax.bar(range(len(words)), word_values, color=(0.1, 0.1, 0.1, 0.1),  edgecolor='blue')
ax.set_xticks(range(len(words)))
ax.set_xticklabels(words, rotation='vertical')
ax.set_title('Top words in new articles dataset (excluding stop words)')
ax.set_xlabel('Words')
ax.set_ylabel('Number of occurences')
ax.patch.set_facecolor('#f0ffff')
plt.show()

# **Stemming/Lemmatizing the data**


In [ ]:
# contents of articles
reindexed_data

In [ ]:
# downloading wordNet lemmatizer
nltk.download(['wordnet','omw-1.4'])
lemmatizer = WordNetLemmatizer()

In [ ]:
# lemmatizing an article to see what wordnet returns
for rows in reindexed_data:
 print(rows)
 test = [lemmatizer.lemmatize(words) for words in rows.split(' ')]
 print(test)
 break

# **lemmatization using Snowball Lemmatizer.**

In [ ]:
# making a lemmatizer object
sno = nltk.stem.SnowballStemmer('english')

In [ ]:
# lemmatizing an article to see what snowball lemmatizer returns
for rows in reindexed_data:
 print(rows)
 test = [sno.stem(words) for words in rows.split(' ')]
 print(test)
 break


# **lemmatization using TextBlob.**

In [ ]:
# Lemmatizing using textblob
for rows in reindexed_data:
 print(rows)
 test = [Word(words).lemmatize() for words in rows.split(' ')]
 print(test)
 break

### What all manipulations have you done and insights you found?

We can see that Business and Sports category have the highest number of articles, while for Tech category the number is quite low. Number of articles will play a major role in determining the topics in them. Larger the number of articles for a category, higher will be the result satisfaction for the model.

## ***6. Feature Engineering & Data Pre-processing***

### 4. Textual Data Preprocessing
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

# ***Feature Extraction (Vectorization)***

In [ ]:
# creating a countvectorizer object
count_vectorizer = CountVectorizer(stop_words = stpwrd, max_features = 4000)

# text before vectorization
text_sample = reindexed_data
print('Content before vectorization: {}'.format(text_sample[121]))

# encode the textual content
document_term_matrix = count_vectorizer.fit_transform(text_sample)

# text after vectorization
print('Content after vectorization: \n{}'.format(document_term_matrix[121]))


## ***7. ML Model Implementation***

ML Model - 1
Latent Dirichlet Allocation (LDA)

In [ ]:
# hyperparameter tuning using Grid Search CV
# Parameters tuning using Grid Search
from sklearn.model_selection import GridSearchCV
grid_params = {'n_components' : list(range(5,10))}

# LDA model
lda = LatentDirichletAllocation()
lda_model = GridSearchCV(lda,param_grid=grid_params)
lda_model.fit(document_term_matrix)

# Best LDA model
best_lda_model = lda_model.best_estimator_

print("Best LDA model's params" , lda_model.best_params_)
print("Best log likelihood Score for the LDA model",lda_model.best_score_)
print("LDA model Perplexity on train data", best_lda_model.perplexity(document_term_matrix))

In [ ]:
# importing pyLDAvis module
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
# generate and display the graph
lda_panel = pyLDAvis.sklearn.prepare(best_lda_model, document_term_matrix, count_vectorizer, mds='tsne')
lda_panel

In [ ]:
# creating docterms dataframe
docterms = lda_panel.token_table.sort_values(by = ['Freq'], ascending=False)

In [ ]:
# display docterms df
docterms

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

# Creating Dataframe to contain top 50 terms by topic.

In [ ]:
# create topics dataframe
topicsdf = pd.DataFrame()
# adding top 50 most relevant terms for each topic to the dataframe
for i in range(1,6):
  Topicdict ={ "Topic":i, "Terms":list(docterms[docterms['Topic']==i]['Term'].head(50))  }
  topicsdf=topicsdf.append(Topicdict,ignore_index=True)
topicsdf

In [ ]:
# Visualizing evaluation Metric Score chart

# ***Topic 1: Tech***

In [ ]:
# creating term freq dict for topic 1
t1dict = {}
for vals in docterms[docterms['Topic']==1].head(40).values:
  t1dict[vals[2]] =vals[1]
t1dict

In [ ]:
# Generating the Wordcloud for topic 1
wordcloud = WordCloud(width = 1200, height = 700,
                background_color ='beige',
                min_font_size = 10).generate(' '.join(list(t1dict.keys())))
wordcloud = wordcloud.generate_from_frequencies(frequencies=t1dict)

In [ ]:
# plotting the WordCloud image
plt.figure(figsize = (12,7), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

# Topic 2: **Business**

In [ ]:
#creating term freq dict for topic 2
t2dict = {}
for vals in docterms[docterms['Topic']==2].head(40).values:
  t2dict[vals[2]] =vals[1]
t2dict

In [ ]:
# generating the wordcloud for topic 2
wordcloud = WordCloud(width = 1200, height = 700,
                background_color ='black',
                min_font_size = 10).generate(' '.join(list(t2dict.keys())))
wordcloud = wordcloud.generate_from_frequencies(frequencies=t2dict)

In [ ]:
# plotting the WordCloud image
plt.figure(figsize = (12,7), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

# **Topic 3: Sports**

In [ ]:
# creating term freq dict for topic 3
t3dict = {}
for vals in docterms[docterms['Topic']==3].head(40).values:
  t3dict[vals[2]] =vals[1]
t3dict

In [ ]:
# generating the Wordcloud for topic 3
wordcloud = WordCloud(width = 1200, height = 700,
                background_color ='magenta',
                min_font_size = 10).generate(' '.join(list(t3dict.keys())))
wordcloud = wordcloud.generate_from_frequencies(frequencies=t3dict)

In [ ]:
# plotting the WordCloud image
plt.figure(figsize = (12,7), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

# ***Topic 4: Politics***

In [ ]:
# creating term freq dict for topic 4
t4dict = {}
for vals in docterms[docterms['Topic']==4].head(40).values:
  t4dict[vals[2]] =vals[1]
t4dict

In [ ]:
# generating the wordcloud for topic 4
wordcloud = WordCloud(width = 1200, height = 700,
                background_color ='cyan',
                min_font_size = 10).generate(' '.join(list(t4dict.keys())))
wordcloud = wordcloud.generate_from_frequencies(frequencies=t4dict)

In [ ]:
# plotting the WordCloud image
plt.figure(figsize = (12,7), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

# ***Topic 5: Entertainment***

In [ ]:
# creating term freq dict for topic 5
t5dict = {}
for vals in docterms[docterms['Topic']==5].head(40).values:
  t5dict[vals[2]] =vals[1]
t5dict

In [ ]:
# generating the wordcloud for topic 5
wordcloud = WordCloud(width = 1200, height = 700,
                background_color ='maroon',
                min_font_size = 10).generate(' '.join(list(t5dict.keys())))
wordcloud = wordcloud.generate_from_frequencies(frequencies=t5dict)

In [ ]:
# plotting the WordCloud image
plt.figure(figsize = (12,7), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***